In [1]:
from PIL import Image, ImageDraw,ImageChops
import numpy as np
from os import listdir
from os.path import isfile, join
import sys
import time
from datetime import datetime

baseDir = '/home/mick/Desktop/gpufilesystem/'
dataDir = baseDir + 'dataset/'
optDir = baseDir + 'examples/GPU/opt/'
sys.path.insert(1, baseDir)
from lib import pyGPUfs

#delete all the previous files in the opt dir
files = [f for f in listdir(optDir) if isfile(join(optDir, f))]
for file in files:
    import os
    os.remove(optDir + file)
    
#load all the files in the dataset dir to GPU
files = [f for f in listdir(dataDir) if isfile(join(dataDir, f))]
for file in files:
    pyGPUfs.store(dataDir + file)

In [2]:
#pyGPUfs.list()
pyGPUfs.findByName('100')
pyGPUfs.readByIndex(0, optDir + '100.jpg')

pyGPUfs.writeByIndex(0, dataDir + '0')
pyGPUfs.findByName('100')

findByName(100)
0 	File Name:		100
	UUID:			2aea5b401ef748cd9615d2bac027e78a
	Path:			/home/mick/Desktop/gpufilesystem/dataset/100
	Size:(Byte)		(128, 128, 3)
	Last Access Time:	2020-11-17 21:29:26
	Last Modify Time:	2020-11-17 21:28:23

1 	File Name:		1004
	UUID:			36fa0f0f20fc486a929a462d90634766
	Path:			/home/mick/Desktop/gpufilesystem/dataset/1004
	Size:(Byte)		(128, 128, 3)
	Last Access Time:	2020-11-17 21:29:26
	Last Modify Time:	2020-11-17 21:28:30

2 	File Name:		1300
	UUID:			6813eec16c2847d39f2ced3f1a61e930
	Path:			/home/mick/Desktop/gpufilesystem/dataset/1300
	Size:(Byte)		(128, 128, 3)
	Last Access Time:	2020-11-17 21:29:26
	Last Modify Time:	2020-11-17 21:28:32

3 	File Name:		103
	UUID:			4620b45d87014737ae319bd7f3be1042
	Path:			/home/mick/Desktop/gpufilesystem/dataset/103
	Size:(Byte)		(128, 128, 3)
	Last Access Time:	2020-11-17 21:29:26
	Last Modify Time:	2020-11-17 21:28:23

4 	File Name:		500
	UUID:			358a9631bbb2414c8d772eb9fc5f7f92
	Path:			/home/mick/Desktop/gpu

In [3]:
from lib import Domain
import pycuda.gpuarray as gpuarray
def crop(input, height, width):
    opt = []
    k = 0
    im = input
    imgwidth, imgheight = im.size
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            box = (j, i, j+width, i+height)
            draw = ImageDraw.Draw(im)
            draw.rectangle(box)
            a = im.crop(box)
            a.save(optDir + str(k) + '.jpg')
            k+=1
            opt.append(a)
    im.show()
    return opt

def get_concat_v_multi_resize(im_list, resample=Image.BICUBIC):
    min_width = min(im.width for im in im_list)
    im_list_resize = [im.resize((min_width, int(im.height * min_width / im.width)),resample=resample)
                      for im in im_list]
    total_height = sum(im.height for im in im_list_resize)
    dst = Image.new('RGB', (min_width, total_height))
    pos_y = 0
    for im in im_list_resize:
        dst.paste(im, (0, pos_y))
        pos_y += im.height
    return dst

def get_concat_h_multi_resize(im_list, resample=Image.BICUBIC):
    min_height = min(im.height for im in im_list)
    im_list_resize = [im.resize((int(im.width * min_height / im.height), min_height),resample=resample)
                      for im in im_list]
    total_width = sum(im.width for im in im_list_resize)
    dst = Image.new('RGB', (total_width, min_height))
    pos_x = 0
    for im in im_list_resize:
        dst.paste(im, (pos_x, 0))
        pos_x += im.width
    return dst

def get_concat_tile_resize(im_list_2d, resample=Image.BICUBIC):
    im_list_v = [get_concat_h_multi_resize(im_list_h, resample=resample) for im_list_h in im_list_2d]
    return get_concat_v_multi_resize(im_list_v, resample=resample)

start = time.time()
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time :", current_time)

#the path of the target image
target_path = dataDir + '4'

#the dim of the output image collage, dim[0] and dim[1] have to be of the power of 2
dim = (32, 32, 3)

im = Image.open(target_path)
im = im.resize((1024,1024))

matrix_width = int(im.size[0] / dim[0])
matrix_height = int(im.size[1] / dim[1])

image_segments = crop(im, matrix_width, matrix_height)

vertical = []
for x in range(dim[0]):
    horizontal = []
    for y in range(dim[1]):
        target_index = x * dim[0] + y
        #print(x,y,target_index, end = ', ')
        print('.', end = '')
        diff_list = []
        target = Image.open(optDir + str(target_index) + '.jpg')
        target = target.resize((128,128))
        a = np.array(target)
        a = a.astype('float64')
        gpuarrayInst = gpuarray.to_gpu(a)
        size = (a.shape)
        targetFile = (Domain.pyGPUfsFile(
            fileName = 'target', 
            absPath = 'path', 
            nbyte = 'nbyte', 
            fileType = 'image', 
            gpuarray = gpuarrayInst, 
            size = size, 
            lastAccessTime = 'lastAccessTime',
            lastModifyTime = 'lastModifyTime'))
        #target.show()
        for file_index in range(len(pyGPUfs.Global.fileList)):
            candidate = pyGPUfs.Global.fileList[file_index]
            diff = pyGPUfs.compare(targetFile, candidate)
            diff_list.append(int(diff))
        path = dataDir + pyGPUfs.Global.fileList[diff_list.index(min(diff_list))].fileName
        result = Image.open(path)
        horizontal.append(result)
    vertical.append(horizontal)
opt_img = get_concat_tile_resize(vertical)
opt_img = opt_img.resize((1024,1024))
opt_img.save('opt.jpg')
opt_img.show()

done = time.time()
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Finish Time :", current_time)
elapsed = done - start
print('Processing used (s): ', elapsed)

Start Time : 18:18:25
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................